In [ ]:
import os #ceate folders
from glob import glob # get paths and use the to have floders name
from tqdm.notebook import tqdm # get nice bar
import sys
import pydicom as pdc  # read dicom images
import numpy as np
import imageio    # save to PNG images
import matplotlib.pyplot as plt  # plot some PNG images
import cv2 as cv  # read PNG images
from random import sample 
from joblib import Parallel,delayed
import subprocess
from ast import literal_eval



In [ ]:
# get CPU informations 

def run(command):
    process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE)
    out, err = process.communicate()
    print(out.decode('utf-8').strip())
    
print('# CPU')
run('cat /proc/cpuinfo | egrep -m 1 "^model name"')
run('cat /proc/cpuinfo | egrep -m 1 "^cpu MHz"')
run('cat /proc/cpuinfo | egrep -m 1 "^cpu cores"')

In [ ]:
#train and test folder names 
train_path_list=glob('../input/rsna-miccai-brain-tumor-radiogenomic-classification/train/*')
test_path_list= glob('../input/rsna-miccai-brain-tumor-radiogenomic-classification/test/*')
kaggle_input_path= '../input/'

#list of name of subdirectorys
train_d=list(map(lambda path:path.split('/')[-1],train_path_list))
test_d=list(map(lambda path:path.split('/')[-1],test_path_list))
mpMRI_scans=["FLAIR","T1w","T1wCE","T2w"] 

# sample of names
print(train_d[:4])

In [ ]:
# read dico image and return the image array

def read_dcm(img):
    """reading a dicom image with preprocessing"""
    dcm_img=pdc.dcmread(img)
    img_array=dcm_img.pixel_array
    img_array = img_array - np.min(img_array)
    if np.max(img_array) != 0:
        img_array = img_array / np.max(img_array)
    img_array = (img_array * 255).astype(np.uint8)
    return img_array

In [ ]:
#delete old folder (if you run the code twice)

!rm -rf  Png-rsna-miccai-brain-tumor-radiogenomic-classification/

# create the main PNG folder with the test and train 

png_test_path='Png-rsna-miccai-brain-tumor-radiogenomic-classification/test'
png_train_path='Png-rsna-miccai-brain-tumor-radiogenomic-classification/train'

os.mkdir('Png-rsna-miccai-brain-tumor-radiogenomic-classification')
os.mkdir(png_train_path)
os.mkdir(png_test_path)


# floders creation 

for trfold in train_d:
    os.mkdir(png_train_path+'/'+trfold)
    for mp in mpMRI_scans:
        os.mkdir(png_train_path+'/'+trfold+'/'+mp)
        
for tsfold in test_d: 
    os.mkdir(png_test_path+'/'+tsfold)
    for mp in mpMRI_scans:
        os.mkdir(png_test_path+'/'+tsfold+'/'+mp)

In [ ]:
# list the paths and name directory 

png_train_path_list=glob('Png-rsna-miccai-brain-tumor-radiogenomic-classification/test/*')
png_test_path_list= glob('Png-rsna-miccai-brain-tumor-radiogenomic-classification/train/*')

png_train_d=list(map(lambda path:path.split('/')[-1],png_train_path_list))
png_test_d=list(map(lambda path:path.split('/')[-1],png_test_path_list))

#print sample of Png path folders and there names
print(png_train_path_list[:3])
print(png_train_d[:3])


# compare the folder result 

print(train_d.sort()==png_train_d.sort())
print(test_d.sort()==png_test_d.sort())

# list of all dicom iamges paths
train_images_path=glob('../input/rsna-miccai-brain-tumor-radiogenomic-classification/train/*/*/*')
test_images_path=glob('../input/rsna-miccai-brain-tumor-radiogenomic-classification/test/*/*/*')


In [ ]:
# the logic to create the new PNG path 

image_name=test_images_path[0].split('/')[4:][-1].split('.')[0]
new_png_path=png_train_path+'/'+'/'.join(train_images_path[0].split('/')[4:-1])+'/'+image_name+'.PNG'
print(new_png_path)


In [ ]:
print(len(train_images_path),len(test_images_path))


In [ ]:
# PNG save function 

def train_png(train_path):
    train_array_img=read_dcm(train_path)                                             
    train_image_name=train_path.split('/')[4:][-1].split('.')[0]                               
    new_train_png_path=png_train_path+'/'+'/'.join(train_path.split('/')[4:-1])+'/'+train_image_name+'.PNG'    
    imageio.imsave(new_train_png_path,train_array_img)
    
def test_png(test_path):
    test_array_path=read_dcm(test_path) 
    test_image_name=test_path.split('/')[4:][-1].split('.')[0]
    new_test_png_path=png_test_path+'/'+'/'.join(test_path.split('/')[4:-1])+'/'+test_image_name+'.PNG'
    imageio.imsave(new_test_png_path,test_array_path)

In [ ]:
#FIXED : not all the training data was converted and the slow conversion 
# test for 20000 for tarain and 10000 for test 
fb_train=Parallel(n_jobs=4,verbose=1,prefer='threads')(delayed(train_png)(train_path) for train_path in tqdm(train_images_path[:20000],total=len(train_images_path[:20000])))
fb_test=Parallel(n_jobs=4,verbose=1,prefer='threads') (delayed(test_png) (test_path) for test_path in tqdm(test_images_path[:20000],total=len(test_images_path[:20000])))


### Image creation and saving part

In [ ]:
# FIXED : printing some black images 

#print(PNG random PNG images)
png_images=glob('Png-rsna-miccai-brain-tumor-radiogenomic-classification/test/*/*/*')
print(png_images[:10])

plt.figure(figsize=(15,15))
plot_indicator=0
sub_in=0
for index,image in tqdm(enumerate(sample(png_images,5000))) :
    img = cv.imread(image,cv.IMREAD_GRAYSCALE)
    img=cv.resize(img,(200,200))
    if np.max(img)!= 0 and np.mean(img)>=30:  
        plot_indicator+=1
        if plot_indicator==5:
            break
        else :    
            sub_in+=1
            plt.subplot(2,2,sub_in)  
            plt.imshow(img)
            
    else:
        continue 

In [ ]:
!tar -zcf dcm_to_png.tar.gz -C "/kaggle/working/Png-rsna-miccai-brain-tumor-radiogenomic-classification/" .